In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import json

import sys

sys.path.append("../")

##################################################################
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3,4,5,6,7"
##################################################################

import logging
from src.utils import logging_utils
from src.utils import env_utils

logger = logging.getLogger(__name__)

logging.basicConfig(
    level=logging.DEBUG,
    format=logging_utils.DEFAULT_FORMAT,
    datefmt=logging_utils.DEFAULT_DATEFMT,
    stream=sys.stdout,
)

import torch
import transformers

logger.info(f"{torch.__version__=}, {torch.version.cuda=}")
logger.info(
    f"{torch.cuda.is_available()=}, {torch.cuda.device_count()=}, {torch.cuda.get_device_name()=}"
)
logger.info(f"{transformers.__version__=}")

2025-07-30 14:07:23 __main__ INFO     torch.__version__='2.7.0+cu126', torch.version.cuda='12.6'
2025-07-30 14:07:23 __main__ INFO     torch.cuda.is_available()=True, torch.cuda.device_count()=8, torch.cuda.get_device_name()='NVIDIA A100 80GB PCIe'
2025-07-30 14:07:23 __main__ INFO     transformers.__version__='4.51.3'


In [3]:
from src.utils.training_utils import get_device_map

# model_key = "meta-llama/Llama-3.2-3B"
# model_key = "meta-llama/Llama-3.1-8B"
model_key = "meta-llama/Llama-3.3-70B-Instruct"
# model_key = "meta-llama/Llama-3.1-405B-Instruct"

# model_key = "google/gemma-2-9b-it"
# model_key = "google/gemma-3-12b-it"
# model_key = "google/gemma-2-27b-it"

# model_key = "deepseek-ai/DeepSeek-R1-Distill-Llama-8B"

# model_key = "allenai/OLMo-2-1124-7B-Instruct"
# model_key = "allenai/OLMo-7B-0424-hf"

# model_key = "Qwen/Qwen2-7B"
# model_key = "Qwen/Qwen2.5-14B-Instruct"
# model_key = "Qwen/Qwen2.5-32B-Instruct"
# model_key = "Qwen/Qwen2.5-72B-Instruct"

# model_key = "Qwen/Qwen3-1.7B"
# model_key = "Qwen/Qwen3-4B"
# model_key = "Qwen/Qwen3-8B"
# model_key = "Qwen/Qwen3-14B"
# model_key = "Qwen/Qwen3-32B"

# device_map = get_device_map(model_key, 30, n_gpus=8)
# device_map

[2025-07-30 14:07:25,970] [INFO] [real_accelerator.py:239:get_accelerator] Setting ds_accelerator to cuda (auto detect)
2025-07-30 14:07:26 root INFO     gcc -pthread -B /disk/u/arnab/miniconda3/envs/connection/compiler_compat -DNDEBUG -fwrapv -O2 -Wall -fPIC -O2 -isystem /disk/u/arnab/miniconda3/envs/connection/include -fPIC -O2 -isystem /disk/u/arnab/miniconda3/envs/connection/include -fPIC -c /tmp/tmp8osoh7hz/test.c -o /tmp/tmp8osoh7hz/test.o
2025-07-30 14:07:26 root INFO     gcc -pthread -B /disk/u/arnab/miniconda3/envs/connection/compiler_compat /tmp/tmp8osoh7hz/test.o -laio -o /tmp/tmp8osoh7hz/a.out


/disk/u/arnab/miniconda3/envs/connection/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status


2025-07-30 14:07:26 root INFO     gcc -pthread -B /disk/u/arnab/miniconda3/envs/connection/compiler_compat -DNDEBUG -fwrapv -O2 -Wall -fPIC -O2 -isystem /disk/u/arnab/miniconda3/envs/connection/include -fPIC -O2 -isystem /disk/u/arnab/miniconda3/envs/connection/include -fPIC -c /tmp/tmpqxscc_rm/test.c -o /tmp/tmpqxscc_rm/test.o
2025-07-30 14:07:26 root INFO     gcc -pthread -B /disk/u/arnab/miniconda3/envs/connection/compiler_compat /tmp/tmpqxscc_rm/test.o -L/usr -L/usr/lib64 -lcufile -o /tmp/tmpqxscc_rm/a.out


/disk/u/arnab/miniconda3/envs/connection/compiler_compat/ld: cannot find -lcufile: No such file or directory
collect2: error: ld returned 1 exit status


2025-07-30 14:07:27 git.cmd DEBUG    Popen(['git', 'version'], cwd=/disk/u/arnab/Codes/Projects/retrieval/notebooks, stdin=None, shell=False, universal_newlines=False)
2025-07-30 14:07:27 git.cmd DEBUG    Popen(['git', 'version'], cwd=/disk/u/arnab/Codes/Projects/retrieval/notebooks, stdin=None, shell=False, universal_newlines=False)
2025-07-30 14:07:27 wandb.docker.auth DEBUG    Trying paths: ['/disk/u/arnab/.docker/config.json', '/disk/u/arnab/.dockercfg']
2025-07-30 14:07:27 wandb.docker.auth DEBUG    No config file found


In [4]:
# os.environ["BNB_CUDA_VERSION"] = "124"
# ! echo $BNB_CUDA_VERSION
# ! python -m bitsandbytes

In [5]:
from src.models import ModelandTokenizer

# from transformers import BitsAndBytesConfig

mt = ModelandTokenizer(
    model_key=model_key,
    torch_dtype=torch.bfloat16,
    # device_map=device_map,
    device_map="auto",
    # quantization_config = BitsAndBytesConfig(
    #     # load_in_4bit=True
    #     load_in_8bit=True
    # )
)

2025-07-30 14:07:32 src.models WARNING  meta-llama/Llama-3.3-70B-Instruct not found in /disk/u/arnab/Codes/Models
If not found in cache, model will be downloaded from HuggingFace to cache directory
2025-07-30 14:07:32 urllib3.connectionpool DEBUG    Starting new HTTPS connection (1): huggingface.co:443
2025-07-30 14:07:32 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /meta-llama/Llama-3.3-70B-Instruct/resolve/main/config.json HTTP/1.1" 200 0
2025-07-30 14:07:32 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /meta-llama/Llama-3.3-70B-Instruct/resolve/main/tokenizer_config.json HTTP/1.1" 200 0


Loading checkpoint shards:   0%|          | 0/30 [00:00<?, ?it/s]

2025-07-30 14:08:21 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /meta-llama/Llama-3.3-70B-Instruct/resolve/main/generation_config.json HTTP/1.1" 200 0
2025-07-30 14:08:21 src.models INFO     loaded model <meta-llama/Llama-3.3-70B-Instruct> | size: 134570.516 MB | dtype: torch.bfloat16 | device: cuda:0


In [6]:
from src.functional import free_gpu_cache

# SYNTH_DATASET = "icosahedron_1"
SYNTH_DATASET = "64"

checkpoint_path = os.path.join(
    env_utils.DEFAULT_RESULTS_DIR,
    "trained_params",
    f"{SYNTH_DATASET}",
    "_full__clamp=0.001",
    model_key.split("/")[-1],
)

version = "epoch_1"
# version = "final_model"

checkpoint_path = os.path.join(env_utils.DEFAULT_RESULTS_DIR, checkpoint_path, version)

print(os.listdir(checkpoint_path))

checkpoint_path = os.path.join(checkpoint_path, "trainable_params.pt")

loaded_deltas = torch.load(checkpoint_path, map_location="cpu")
# loaded_deltas

free_gpu_cache()


d = loaded_deltas["model<>layers<>5<>mlp<>gate_proj"]
d.abs().max()

['trainable_params.pt']


tensor(0.0010, dtype=torch.bfloat16, grad_fn=<MaxBackward1>)

In [7]:
from src.utils.training_utils import TrainableLM_delta, TrainableLM_LoRA

#################################################
Trainable_CLS = TrainableLM_delta
# Trainable_CLS = TrainableLM_LoRA
#################################################

Trainable_CLS.fuse_with_model(mt._model, loaded_deltas)

2025-07-30 14:08:28 src.utils.training_utils DEBUG    module_name='model.layers.0.mlp.gate_proj' | param_delta.shape=torch.Size([28672, 8192])
2025-07-30 14:08:29 src.utils.training_utils DEBUG    module_name='model.layers.0.mlp.up_proj' | param_delta.shape=torch.Size([28672, 8192])
2025-07-30 14:08:29 src.utils.training_utils DEBUG    module_name='model.layers.0.mlp.down_proj' | param_delta.shape=torch.Size([8192, 28672])
2025-07-30 14:08:29 src.utils.training_utils DEBUG    module_name='model.layers.1.mlp.gate_proj' | param_delta.shape=torch.Size([28672, 8192])
2025-07-30 14:08:29 src.utils.training_utils DEBUG    module_name='model.layers.1.mlp.up_proj' | param_delta.shape=torch.Size([28672, 8192])
2025-07-30 14:08:29 src.utils.training_utils DEBUG    module_name='model.layers.1.mlp.down_proj' | param_delta.shape=torch.Size([8192, 28672])
2025-07-30 14:08:29 src.utils.training_utils DEBUG    module_name='model.layers.2.mlp.gate_proj' | param_delta.shape=torch.Size([28672, 8192])
202

In [70]:
###############################################################################
LAYERS = [35, 47, 52]
HEADS = [
    (33, 45),
    (33, 18),
    (34, 1),
    (34, 6),
    (34, 7),
    (35, 19),
    (39, 40),
    (42, 30),
    (47, 18),
    (52, 58),
]
###############################################################################

In [64]:
from src.selection.data  import load_people_by_category

people_by_category = load_people_by_category(tokenizer = mt.tokenizer)
list(people_by_category.keys())

2025-07-30 15:13:54 src.selection.data INFO     Loaded 16 categories


['actor',
 'singer',
 'comedian',
 'director',
 'basketball player',
 'football player',
 'soccer player',
 'tennis player',
 'golfer',
 'boxer',
 'news anchor',
 'journalist',
 'author',
 'fashion designer',
 'entrepreneur',
 'politician']

In [135]:
from src.selection.data import get_random_sample

sample = get_random_sample(
    people_by_category=people_by_category,
    mt=mt,
    attribute="actor",
    n_distractors=5
)
print(sample.prompt, ">>", sample.obj)

Which person from the following list has their occupation in common with Samuel L. Jackson?
Options: Norah O'Donnell, Julia Roberts, Rory McIlroy, Petra Kvitová, Christian Louboutin, Wes Anderson.
Ans: >> Julia Roberts


In [140]:
from src.functional import generate_with_patch, predict_next_token

# prompt_template = """Which of these peoples is by profession a {}?
# options: {}
# Answer:"""
# prompt = prompt_template.format(
#     sample.metadata["attribute"],
#     ", ".join(sample.options)
# )

prompt_template = """{}
Who among these people mentioned above is by profession a {}?
Answer:"""
prompt = prompt_template.format(
    ", ".join(sample.options),
    sample.metadata["attribute"]
)

# prompt = sample.prompt

print(prompt, " >> ")

gen = generate_with_patch(
    mt=mt,
    inputs=prompt,
    n_gen_per_prompt=1,
    max_new_tokens=20,
    remove_prefix=True,
)[0]
print(gen)

predict_next_token(
    mt = mt,
    inputs=prompt,
)

Norah O'Donnell, Julia Roberts, Rory McIlroy, Petra Kvitová, Christian Louboutin, Wes Anderson
Who among these people mentioned above is by profession a actor?
Answer:  >> 
 Julia Roberts is the actor among the listed individuals.


[[PredictedToken(token=' Julia', prob=0.9453125, logit=22.25, token_id=40394, metadata=None),
  PredictedToken(token=' The', prob=0.017333984375, logit=18.25, token_id=578, metadata=None),
  PredictedToken(token=' Nor', prob=0.010498046875, logit=17.75, token_id=8170, metadata=None),
  PredictedToken(token=' Among', prob=0.006378173828125, logit=17.25, token_id=22395, metadata=None),
  PredictedToken(token=' ', prob=0.0030059814453125, logit=16.5, token_id=220, metadata=None)]]

In [141]:
from src.attention import get_attention_matrices, visualize_average_attn_matrix
from src.functional import interpret_logits
from src.tokens import prepare_input

tokenized = prepare_input(
    tokenizer=mt,
    prompts=prompt
)

attn_matrices = get_attention_matrices(
    input=tokenized,
    mt=mt,
    value_weighted=False,
    # patches = []
)

visualize_average_attn_matrix(
    mt=mt,
    attn_matrices=attn_matrices,
    prompt=prompt,
    tokenized=tokenized,
    layer_window=LAYERS,
)

interpret_logits(tokenizer=mt, logits=attn_matrices.logits)

layer=35


--------------------------------------------------------------------------------
layer=47


--------------------------------------------------------------------------------
layer=52


--------------------------------------------------------------------------------


[PredictedToken(token=' Julia', prob=0.953125, logit=22.375, token_id=40394, metadata=None),
 PredictedToken(token=' The', prob=0.015380859375, logit=18.25, token_id=578, metadata=None),
 PredictedToken(token=' Nor', prob=0.00933837890625, logit=17.75, token_id=8170, metadata=None),
 PredictedToken(token=' Among', prob=0.005645751953125, logit=17.25, token_id=22395, metadata=None),
 PredictedToken(token=' ', prob=0.0026702880859375, logit=16.5, token_id=220, metadata=None)]

In [142]:
from src.attention import visualize_attn_matrix

###############################################################
N_KV_GROUPS = 1  # individual heads
# N_KV_GROUPS = mt.config.num_key_value_heads  # grouped heads
N_HEADS = mt.config.num_attention_heads
LAYER_RANGE = range(mt.n_layer)
###############################################################

group_id_to_head_indices = {
    i: [i * N_KV_GROUPS + j for j in range(N_KV_GROUPS)]
    for i in range(N_HEADS // N_KV_GROUPS)
}

In [143]:
# for group_id in group_id_to_head_indices:
for layer_idx, head_idx in HEADS:
    print(
        f"Layer: {layer_idx}, Head: {head_idx}"
    )

    head_matrix = torch.Tensor(attn_matrices.attention_matrices[layer_idx, head_idx]).squeeze()
    visualize_attn_matrix(
        attn_matrix=head_matrix,
        tokens=[mt.tokenizer.decode(t) for t in tokenized.input_ids[0]],
        q_index=-1,
        start_from=1
    )

Layer: 33, Head: 45


Layer: 33, Head: 18


Layer: 34, Head: 1


Layer: 34, Head: 6


Layer: 34, Head: 7


Layer: 35, Head: 19


Layer: 39, Head: 40


Layer: 42, Head: 30


Layer: 47, Head: 18


Layer: 52, Head: 58


### Different Attribute

In [144]:
from src.selection.data  import load_people_by_category

people_by_nationality = load_people_by_category(
    tokenizer = mt.tokenizer,
    path = os.path.join(env_utils.DEFAULT_DATA_DIR, "selection_real/nationality.json")
)
list(people_by_nationality.keys())

2025-07-30 16:35:40 src.selection.data INFO     Loaded 16 categories


['Switzerland',
 'UAE',
 'Singapore',
 'Canada',
 'Mexico',
 'Brazil',
 'India',
 'Australia',
 'Israel',
 'South Africa',
 'United Kingdom',
 'Trinidad and Tobago',
 'New Zealand',
 'United States',
 'Belgium',
 'France']

In [149]:
from src.selection.data import get_random_sample

sample = get_random_sample(
    people_by_category=people_by_nationality,
    mt=mt,
    category="nationality",
    attribute="Australia",
    n_distractors=5
)
print(sample.prompt, ">>", sample.obj)

Which person from the following list has their nationality in common with Kylie Minogue?
Options: Goh Chok Tong, AB de Villiers, Scarlett Johansson, Diego Rivera, Ed Sheeran, Heath Ledger.
Ans: >> Heath Ledger


In [150]:
from src.functional import generate_with_patch, predict_next_token

# prompt_template = """Which of these peoples is by profession a {}?
# options: {}
# Answer:"""
# prompt = prompt_template.format(
#     sample.metadata["attribute"],
#     ", ".join(sample.options)
# )

prompt_template = """{}
Who among these people mentioned above is from {}?
Answer:"""
prompt = prompt_template.format(
    ", ".join(sample.options),
    sample.metadata["attribute"]
)

print(prompt, " >> ")

gen = generate_with_patch(
    mt=mt,
    inputs=prompt,
    n_gen_per_prompt=1,
    max_new_tokens=20,
    remove_prefix=True,
)[0]
print(gen)

predict_next_token(
    mt = mt,
    inputs=prompt,
)

Goh Chok Tong, AB de Villiers, Scarlett Johansson, Diego Rivera, Ed Sheeran, Heath Ledger
Who among these people mentioned above is from Australia?
Answer:  >> 
 Heath Ledger
Explanation: Heath Ledger was an Australian actor, best known for his role as the Joker


[[PredictedToken(token=' Heath', prob=0.74609375, logit=20.5, token_id=48562, metadata=None),
  PredictedToken(token=' AB', prob=0.053955078125, logit=17.875, token_id=14469, metadata=None),
  PredictedToken(token=' The', prob=0.047607421875, logit=17.75, token_id=578, metadata=None),
  PredictedToken(token=' ', prob=0.0255126953125, logit=17.125, token_id=220, metadata=None),
  PredictedToken(token=' Only', prob=0.0155029296875, logit=16.625, token_id=8442, metadata=None)]]

In [151]:
from src.attention import get_attention_matrices, visualize_average_attn_matrix
from src.functional import interpret_logits
from src.tokens import prepare_input

tokenized = prepare_input(
    tokenizer=mt,
    prompts=prompt
)

attn_matrices = get_attention_matrices(
    input=tokenized,
    mt=mt,
    value_weighted=False,
    # patches = []
)

visualize_average_attn_matrix(
    mt=mt,
    attn_matrices=attn_matrices,
    prompt=prompt,
    tokenized=tokenized,
    layer_window=LAYERS,
)

interpret_logits(tokenizer=mt, logits=attn_matrices.logits)

layer=35


--------------------------------------------------------------------------------
layer=47


--------------------------------------------------------------------------------
layer=52


--------------------------------------------------------------------------------


[PredictedToken(token=' Heath', prob=0.76953125, logit=20.625, token_id=48562, metadata=None),
 PredictedToken(token=' AB', prob=0.049072265625, logit=17.875, token_id=14469, metadata=None),
 PredictedToken(token=' The', prob=0.043212890625, logit=17.75, token_id=578, metadata=None),
 PredictedToken(token=' ', prob=0.023193359375, logit=17.125, token_id=220, metadata=None),
 PredictedToken(token=' Two', prob=0.0140380859375, logit=16.625, token_id=9220, metadata=None)]

In [152]:
# for group_id in group_id_to_head_indices:
for layer_idx, head_idx in HEADS:
    print(
        f"Layer: {layer_idx}, Head: {head_idx}"
    )

    head_matrix = torch.Tensor(attn_matrices.attention_matrices[layer_idx, head_idx]).squeeze()
    visualize_attn_matrix(
        attn_matrix=head_matrix,
        tokens=[mt.tokenizer.decode(t) for t in tokenized.input_ids[0]],
        q_index=-1,
        start_from=1
    )

Layer: 33, Head: 45


Layer: 33, Head: 18


Layer: 34, Head: 1


Layer: 34, Head: 6


Layer: 34, Head: 7


Layer: 35, Head: 19


Layer: 39, Head: 40


Layer: 42, Head: 30


Layer: 47, Head: 18


Layer: 52, Head: 58


In [153]:
import random

# options = ["Table", "Orange", "Transistor", "Spinach", "Piano", "Coffee"]
# prompt_template = """Which of these objects is by type a {}?
# options: {}
# Answer:"""
# prompt = prompt_template.format(
#     "musical instrument",
#     ", ".join(options)
# )
# print(prompt)

# options = ["anaconda", "python", "cobra", "viper", "mamba", "rattlesnake"]
# prompt = f"""Which of these snake names is also a programming language?
# options: {", ".join(options)}
# Answer:"""

options = [
    "The Space Needle",
    "Louvre Museum",
    "Colosseum",
    "Christ the Redeemer",
    "State of Liberty",
    "Big Ben",
]
# options += ["Eiffel Tower"]
# random.shuffle(options)
country = "Brazil"
prompt_template = """Which of these landmarks is located in {}?
options:\n{}
Answer:"""
prompt = prompt_template.format(
    country, "\n".join([f"{idx+1}. {opt}" for idx, opt in enumerate(options)])
)

print(prompt)

gen = generate_with_patch(
    mt=mt,
    inputs=prompt,
    n_gen_per_prompt=1,
    max_new_tokens=20,
    remove_prefix=True,
)[0]
print(gen)
# predict_next_token(
#     mt=mt,
#     inputs=prompt,
# )

Which of these landmarks is located in Brazil?
options:
1. The Space Needle
2. Louvre Museum
3. Colosseum
4. Christ the Redeemer
5. State of Liberty
6. Big Ben
Answer:
 4. Christ the Redeemer
Explanation: Christ the Redeemer is a famous landmark located in


In [154]:
from src.attention import get_attention_matrices, visualize_average_attn_matrix
from src.functional import interpret_logits
from src.tokens import prepare_input

tokenized = prepare_input(
    tokenizer=mt,
    prompts=prompt
)

attn_matrices = get_attention_matrices(
    input=tokenized,
    mt=mt,
    value_weighted=False,
    # patches = []
)

visualize_average_attn_matrix(
    mt=mt,
    attn_matrices=attn_matrices,
    prompt=prompt,
    tokenized=tokenized,
    layer_window=LAYERS,
)

interpret_logits(tokenizer=mt, logits=attn_matrices.logits)

layer=35


--------------------------------------------------------------------------------
layer=47


--------------------------------------------------------------------------------
layer=52


--------------------------------------------------------------------------------


[PredictedToken(token=' ', prob=0.6484375, logit=21.125, token_id=220, metadata=None),
 PredictedToken(token=' Christ', prob=0.1640625, logit=19.75, token_id=3771, metadata=None),
 PredictedToken(token=' \n', prob=0.068359375, logit=18.875, token_id=720, metadata=None),
 PredictedToken(token=' The', prob=0.022216796875, logit=17.75, token_id=578, metadata=None),
 PredictedToken(token=' Number', prob=0.0196533203125, logit=17.625, token_id=5742, metadata=None)]

In [155]:
# for group_id in group_id_to_head_indices:
for layer_idx, head_idx in HEADS:
    print(
        f"Layer: {layer_idx}, Head: {head_idx}"
    )

    head_matrix = torch.Tensor(attn_matrices.attention_matrices[layer_idx, head_idx]).squeeze()
    visualize_attn_matrix(
        attn_matrix=head_matrix,
        tokens=[mt.tokenizer.decode(t) for t in tokenized.input_ids[0]],
        q_index=-1,
        start_from=1
    )

Layer: 33, Head: 45


Layer: 33, Head: 18


Layer: 34, Head: 1


Layer: 34, Head: 6


Layer: 34, Head: 7


Layer: 35, Head: 19


Layer: 39, Head: 40


Layer: 42, Head: 30


Layer: 47, Head: 18


Layer: 52, Head: 58


In [ ]:
options = ["Marie Curie", "Albert Einstein", "Grover Cleveland", "Charles Darwin", "Nikola Tesla", "Issac Newton"]
prompt_template = """Which of these people is not a scientist?
options: {}
Answer:"""
prompt = prompt_template.format(
    ", ".join(options)
)
print(prompt)

predict_next_token(
    mt=mt,
    inputs=prompt,
)

tokenized = prepare_input(
    tokenizer=mt,
    prompts=prompt
)

attn_matrices = get_attention_matrices(
    input=tokenized,
    mt=mt,
    value_weighted=False,
    # patches = []
)

visualize_average_attn_matrix(
    mt=mt,
    attn_matrices=attn_matrices,
    prompt=prompt,
    tokenized=tokenized,
    layer_window=LAYERS,
)

interpret_logits(tokenizer=mt, logits=attn_matrices.logits)

Which of these people is not a scientist?
options: Marie Curie, Albert Einstein, Grover Cleveland, Charles Darwin, Nikola Tesla, Issac Newton
Answer:
layer=35


--------------------------------------------------------------------------------
layer=47


--------------------------------------------------------------------------------
layer=52


--------------------------------------------------------------------------------


[PredictedToken(token=' Gro', prob=0.96875, logit=21.5, token_id=18370, metadata=None),
 PredictedToken(token=' The', prob=0.00653076171875, logit=16.5, token_id=578, metadata=None),
 PredictedToken(token=' ', prob=0.002716064453125, logit=15.625, token_id=220, metadata=None),
 PredictedToken(token='Gro', prob=0.0023956298828125, logit=15.5, token_id=77435, metadata=None),
 PredictedToken(token=' the', prob=0.00154876708984375, logit=15.0625, token_id=279, metadata=None)]

In [ ]:
for layer_idx, head_idx in HEADS:
    print(
        f"Layer: {layer_idx}, Head: {head_idx}"
    )

    head_matrix = torch.Tensor(attn_matrices.attention_matrices[layer_idx, head_idx]).squeeze()
    visualize_attn_matrix(
        attn_matrix=head_matrix,
        tokens=[mt.tokenizer.decode(t) for t in tokenized.input_ids[0]],
        q_index=-1,
        start_from=1
    )

Layer: 34, Head: 1


Layer: 35, Head: 19


Layer: 39, Head: 40


Layer: 47, Head: 17


Layer: 47, Head: 18


Layer: 52, Head: 19


Layer: 52, Head: 58


In [172]:
options = ["Michael Jordan", "Serena Williams", "Justin Trudeau", "Mike Tyson", "Carl Sagan", "Tom Cruise"]
prompt_template = """Which of these people is a politician or a scientist?
options: {}
Answer:"""

prompt = prompt_template.format(
    ", ".join(options)
)
print(prompt)
gen = generate_with_patch(
    mt=mt,
    inputs=prompt,
    n_gen_per_prompt=1,
    max_new_tokens=20,
    remove_prefix=True,
)[0]
print(gen)

tokenized = prepare_input(
    tokenizer=mt,
    prompts=prompt
)

attn_matrices = get_attention_matrices(
    input=tokenized,
    mt=mt,
    value_weighted=False,
    # patches = []
)

visualize_average_attn_matrix(
    mt=mt,
    attn_matrices=attn_matrices,
    prompt=prompt,
    tokenized=tokenized,
    layer_window=LAYERS,
)

interpret_logits(tokenizer=mt, logits=attn_matrices.logits)

Which of these people is a politician or a scientist?
options: Michael Jordan, Serena Williams, Justin Trudeau, Mike Tyson, Carl Sagan, Tom Cruise
Answer:
 Justin Trudeau
Explanation: Justin Trudeau is a politician who serves as the Prime Minister of Canada. The
layer=35


--------------------------------------------------------------------------------
layer=47


--------------------------------------------------------------------------------
layer=52


--------------------------------------------------------------------------------


[PredictedToken(token=' Justin', prob=0.859375, logit=20.5, token_id=23278, metadata=None),
 PredictedToken(token=' The', prob=0.033447265625, logit=17.25, token_id=578, metadata=None),
 PredictedToken(token=' Carl', prob=0.0157470703125, logit=16.5, token_id=22770, metadata=None),
 PredictedToken(token=' ', prob=0.0074462890625, logit=15.75, token_id=220, metadata=None),
 PredictedToken(token=' A', prob=0.00616455078125, logit=15.5625, token_id=362, metadata=None)]

In [173]:
for layer_idx, head_idx in HEADS:
    print(
        f"Layer: {layer_idx}, Head: {head_idx}"
    )

    head_matrix = torch.Tensor(attn_matrices.attention_matrices[layer_idx, head_idx]).squeeze()
    visualize_attn_matrix(
        attn_matrix=head_matrix,
        tokens=[mt.tokenizer.decode(t) for t in tokenized.input_ids[0]],
        q_index=-1,
        start_from=1
    )

Layer: 33, Head: 45


Layer: 33, Head: 18


Layer: 34, Head: 1


Layer: 34, Head: 6


Layer: 34, Head: 7


Layer: 35, Head: 19


Layer: 39, Head: 40


Layer: 42, Head: 30


Layer: 47, Head: 18


Layer: 52, Head: 58


In [166]:
options = ["48", "57", "55", "62", "39"]
prompt_template = """Which of these numbers is a multiple of 2?
options: {}
Answer:"""
prompt = prompt_template.format(
    ", ".join(options)
)
print(prompt)
gen = generate_with_patch(
    mt=mt,
    inputs=prompt,
    n_gen_per_prompt=1,
    max_new_tokens=20,
    remove_prefix=True,
)[0]
print(gen)
tokenized = prepare_input(
    tokenizer=mt,
    prompts=prompt
)
attn_matrices = get_attention_matrices(
    input=tokenized,
    mt=mt,
    value_weighted=False,
    # patches = []
)
visualize_average_attn_matrix(
    mt=mt,
    attn_matrices=attn_matrices,
    prompt=prompt,
    tokenized=tokenized,
    layer_window=LAYERS,
)
interpret_logits(tokenizer=mt, logits=attn_matrices.logits)

Which of these numbers is a multiple of 2?
options: 48, 57, 55, 62, 39
Answer:
 48
Explanation: A multiple of 2 is an even number, so we need to find
layer=35


--------------------------------------------------------------------------------
layer=47


--------------------------------------------------------------------------------
layer=52


--------------------------------------------------------------------------------


[PredictedToken(token=' ', prob=0.796875, logit=21.375, token_id=220, metadata=None),
 PredictedToken(token=' The', prob=0.07421875, logit=19.0, token_id=578, metadata=None),
 PredictedToken(token=' A', prob=0.034912109375, logit=18.25, token_id=362, metadata=None),
 PredictedToken(token=' Only', prob=0.006072998046875, logit=16.5, token_id=8442, metadata=None),
 PredictedToken(token=' We', prob=0.006072998046875, logit=16.5, token_id=1226, metadata=None)]

In [167]:
# for group_id in group_id_to_head_indices:
for layer_idx, head_idx in HEADS:
    print(
        f"Layer: {layer_idx}, Head: {head_idx}"
    )

    head_matrix = torch.Tensor(attn_matrices.attention_matrices[layer_idx, head_idx]).squeeze()
    visualize_attn_matrix(
        attn_matrix=head_matrix,
        tokens=[mt.tokenizer.decode(t) for t in tokenized.input_ids[0]],
        q_index=-1,
        start_from=1
    )

Layer: 33, Head: 45


Layer: 33, Head: 18


Layer: 34, Head: 1


Layer: 34, Head: 6


Layer: 34, Head: 7


Layer: 35, Head: 19


Layer: 39, Head: 40


Layer: 42, Head: 30


Layer: 47, Head: 18


Layer: 52, Head: 58


In [156]:
# NOVELTY: Find the odd one out

# options = ["Issac Newton", "Albert Einstein", "Marie Curie", "Hugh Jackman", "Charles Darwin"]
options = [
    "Brad Pitt",
    "Tom Hanks",
    "Leonardo DiCaprio",
    "Scarlett Johansson",
    "Daniel Radcliffe",
    "Hugh Jackman",
]

prompt_template = """Which person from the following list is different from the others?
{}.
Ans:"""

prompt = prompt_template.format(", ".join(options))
print(prompt)
gen = generate_with_patch(
    mt=mt,
    inputs=prompt,
    n_gen_per_prompt=1,
    max_new_tokens=20,
    remove_prefix=True,
)[0]
print(gen)
tokenized = prepare_input(tokenizer=mt, prompts=prompt)
attn_matrices = get_attention_matrices(
    input=tokenized,
    mt=mt,
    value_weighted=False,
    # patches = []
)
visualize_average_attn_matrix(
    mt=mt,
    attn_matrices=attn_matrices,
    prompt=prompt,
    tokenized=tokenized,
    layer_window=LAYERS,
)
interpret_logits(tokenizer=mt, logits=attn_matrices.logits)

Which person from the following list is different from the others?
Brad Pitt, Tom Hanks, Leonardo DiCaprio, Scarlett Johansson, Daniel Radcliffe, Hugh Jackman.
Ans:
 Scarlett Johansson. She is the only female in the list. The others are all male actors.
layer=35


--------------------------------------------------------------------------------
layer=47


--------------------------------------------------------------------------------
layer=52


--------------------------------------------------------------------------------


[PredictedToken(token=' Scarlett', prob=0.81640625, logit=19.5, token_id=96572, metadata=None),
 PredictedToken(token=' The', prob=0.031494140625, logit=16.25, token_id=578, metadata=None),
 PredictedToken(token=' Daniel', prob=0.031494140625, logit=16.25, token_id=15469, metadata=None),
 PredictedToken(token=' All', prob=0.01239013671875, logit=15.3125, token_id=2052, metadata=None),
 PredictedToken(token=' ', prob=0.0096435546875, logit=15.0625, token_id=220, metadata=None)]

In [157]:
for layer_idx, head_idx in HEADS:
    print(
        f"Layer: {layer_idx}, Head: {head_idx}"
    )

    head_matrix = torch.Tensor(attn_matrices.attention_matrices[layer_idx, head_idx]).squeeze()
    visualize_attn_matrix(
        attn_matrix=head_matrix,
        tokens=[mt.tokenizer.decode(t) for t in tokenized.input_ids[0]],
        q_index=-1,
        start_from=1
    )

Layer: 33, Head: 45


Layer: 33, Head: 18


Layer: 34, Head: 1


Layer: 34, Head: 6


Layer: 34, Head: 7


Layer: 35, Head: 19


Layer: 39, Head: 40


Layer: 42, Head: 30


Layer: 47, Head: 18


Layer: 52, Head: 58


In [174]:
# rhymes with

options = ["orange", "mat", "book", "rabbit", "bowl", "watch", "mirror"]
# word = "rat"
word = "look"
# word = "fetch"
# word = "growl"

prompt_template = """{}
Which of these words rhymes with the word "{}"?
Answer:"""

prompt = prompt_template.format(
    ", ".join(options),
    word
)
print(prompt)
gen = generate_with_patch(
    mt=mt,
    inputs=prompt,
    n_gen_per_prompt=1,
    max_new_tokens=20,
    remove_prefix=True,
)[0]
print(gen)
tokenized = prepare_input(tokenizer=mt, prompts=prompt)
attn_matrices = get_attention_matrices(
    input=tokenized,
    mt=mt,
    value_weighted=False,
    # patches = []
)
visualize_average_attn_matrix(
    mt=mt,
    attn_matrices=attn_matrices,
    prompt=prompt,
    tokenized=tokenized,
    layer_window=LAYERS,
)
interpret_logits(tokenizer=mt, logits=attn_matrices.logits)

orange, mat, book, rabbit, bowl, watch, mirror
Which of these words rhymes with the word "look"?
Answer:
 book
The word "book" rhymes with the word "look". 
The other options do
layer=35


--------------------------------------------------------------------------------
layer=47


--------------------------------------------------------------------------------
layer=52


--------------------------------------------------------------------------------


[PredictedToken(token=' book', prob=0.65625, logit=20.875, token_id=2363, metadata=None),
 PredictedToken(token=' Book', prob=0.1142578125, logit=19.125, token_id=6017, metadata=None),
 PredictedToken(token=' "', prob=0.0693359375, logit=18.625, token_id=330, metadata=None),
 PredictedToken(token=' The', prob=0.0693359375, logit=18.625, token_id=578, metadata=None),
 PredictedToken(token=' Look', prob=0.00823974609375, logit=16.5, token_id=9372, metadata=None)]

In [175]:
for layer_idx, head_idx in HEADS:
    print(
        f"Layer: {layer_idx}, Head: {head_idx}"
    )

    head_matrix = torch.Tensor(attn_matrices.attention_matrices[layer_idx, head_idx]).squeeze()
    visualize_attn_matrix(
        attn_matrix=head_matrix,
        tokens=[mt.tokenizer.decode(t) for t in tokenized.input_ids[0]],
        q_index=-1,
        start_from=1
    )

Layer: 33, Head: 45


Layer: 33, Head: 18


Layer: 34, Head: 1


Layer: 34, Head: 6


Layer: 34, Head: 7


Layer: 35, Head: 19


Layer: 39, Head: 40


Layer: 42, Head: 30


Layer: 47, Head: 18


Layer: 52, Head: 58


In [158]:
#! Logical deduction

# prompt = """John is taller than Mary. Mary is taller than Steve. Who is the shortest?
# Ans:"""
prompt = """John is taller than Bob, but shorter than Steve. Who is the shortest?
Ans:"""
# prompt = """All the cookies are either chocolate or vanilla. None of the chocolate cookies have nuts. 
# This cookie has nuts. What flavor is it?
# Ans:"""
# prompt = """The red box is to the left of the blue box, and to the right of the green box.
# Which is the rightmost box?
# Ans:"""

gen = generate_with_patch(
    mt=mt,
    inputs=prompt,
    n_gen_per_prompt=1,
    max_new_tokens=50,
    remove_prefix=True,
)[0]
print(f'"{gen}"')

tokenized = prepare_input(tokenizer=mt, prompts=prompt)
attn_matrices = get_attention_matrices(
    input=tokenized,
    mt=mt,
    value_weighted=False,
    # patches = []
)
visualize_average_attn_matrix(
    mt=mt,
    attn_matrices=attn_matrices,
    prompt=prompt,
    tokenized=tokenized,
    layer_window=LAYERS,
)
interpret_logits(tokenizer=mt, logits=attn_matrices.logits)

" Bob
John is taller than Bob, so Bob must be the shortest.
John is shorter than Steve, but that doesn't tell us anything about John's height compared to Bob's. We already know John is taller than Bob, so Bob must be"
layer=35


--------------------------------------------------------------------------------
layer=47


--------------------------------------------------------------------------------
layer=52


--------------------------------------------------------------------------------


[PredictedToken(token=' Bob', prob=0.8359375, logit=20.75, token_id=14596, metadata=None),
 PredictedToken(token=' John', prob=0.036865234375, logit=17.625, token_id=3842, metadata=None),
 PredictedToken(token=' We', prob=0.0196533203125, logit=17.0, token_id=1226, metadata=None),
 PredictedToken(token=' Since', prob=0.0135498046875, logit=16.625, token_id=8876, metadata=None),
 PredictedToken(token=' BO', prob=0.0135498046875, logit=16.625, token_id=7967, metadata=None)]

In [159]:
for layer_idx, head_idx in HEADS:
    print(
        f"Layer: {layer_idx}, Head: {head_idx}"
    )

    head_matrix = torch.Tensor(attn_matrices.attention_matrices[layer_idx, head_idx]).squeeze()
    visualize_attn_matrix(
        attn_matrix=head_matrix,
        tokens=[mt.tokenizer.decode(t) for t in tokenized.input_ids[0]],
        q_index=-1,
        start_from=1
    )

Layer: 33, Head: 45


Layer: 33, Head: 18


Layer: 34, Head: 1


Layer: 34, Head: 6


Layer: 34, Head: 7


Layer: 35, Head: 19


Layer: 39, Head: 40


Layer: 42, Head: 30


Layer: 47, Head: 18


Layer: 52, Head: 58


In [162]:
prompt = """x = True
y = False
z = False

a = x and y
b = y or z
c = z xor x

Which one among a, b, c is True?
Ans:"""

gen = generate_with_patch(
    mt=mt,
    inputs=prompt,
    n_gen_per_prompt=1,
    max_new_tokens=50,
    remove_prefix=True,
)[0]
print(f'"{gen}"')

tokenized = prepare_input(tokenizer=mt, prompts=prompt)
attn_matrices = get_attention_matrices(
    input=tokenized,
    mt=mt,
    value_weighted=False,
    # patches = []
)
visualize_average_attn_matrix(
    mt=mt,
    attn_matrices=attn_matrices,
    prompt=prompt,
    tokenized=tokenized,
    layer_window=LAYERS,
)
interpret_logits(tokenizer=mt, logits=attn_matrices.logits)

" b

Explanation:
a = x and y = True and False = False
b = y or z = False or False = False
c = z xor x = False xor True = True

Among a, b, c only c is True"
layer=35


--------------------------------------------------------------------------------
layer=47


--------------------------------------------------------------------------------
layer=52


--------------------------------------------------------------------------------


[PredictedToken(token=' b', prob=0.69140625, logit=20.5, token_id=293, metadata=None),
 PredictedToken(token=' None', prob=0.09375, logit=18.5, token_id=2290, metadata=None),
 PredictedToken(token=' Only', prob=0.056884765625, logit=18.0, token_id=8442, metadata=None),
 PredictedToken(token=' \n', prob=0.044189453125, logit=17.75, token_id=720, metadata=None),
 PredictedToken(token=' a', prob=0.0184326171875, logit=16.875, token_id=264, metadata=None)]

In [163]:
for layer_idx, head_idx in HEADS:
    print(
        f"Layer: {layer_idx}, Head: {head_idx}"
    )

    head_matrix = torch.Tensor(attn_matrices.attention_matrices[layer_idx, head_idx]).squeeze()
    visualize_attn_matrix(
        attn_matrix=head_matrix,
        tokens=[mt.tokenizer.decode(t) for t in tokenized.input_ids[0]],
        q_index=-1,
        start_from=1
    )

Layer: 33, Head: 45


Layer: 33, Head: 18


Layer: 34, Head: 1


Layer: 34, Head: 6


Layer: 34, Head: 7


Layer: 35, Head: 19


Layer: 39, Head: 40


Layer: 42, Head: 30


Layer: 47, Head: 18


Layer: 52, Head: 58
